# 5. Python Data Wrangling II

_Frederic Hopp and Penny Sheets_

This notebook outlines the retrieval and preprocessing steps we did to construct
the files for our examples. Thus, this notebook contains the steps

- (1) Retrieval
- (2) Preprocessing,

while the other notebook contains the steps

- (3) Enrichment
- (4) Analysis

## EXTRA: How did we prepare the dataset?


### Population data

Go to
https://opendata.cbs.nl/statline/portal.html?_la=en&_catalog=CBS&tableId=37259eng&_theme=1066

Select the following options:

- Topics: Net migration _and_ live born children (2/20)
- Sex: only total (1/3)
- Regions: all provinces (12/1237)

This should give you 984 rows and result in the file

`37259eng_UntypedDataSet_06112024_174233.csv`

Also download the metadata for later reference.

### Economic data

Go to
https://opendata.cbs.nl/statline/portal.html?_la=en&_catalog=CBS&tableId=82800ENG&_theme=1064

Select the following options:

- Topics: GDP (volume change) (1/2)
- Economic sectors: A-U all economic activities (1/15)
- Regions: all provinces (12/77)

This should give you 264 rows and result in the file

`82800ENG_UntypedDataSet_06112024_174453.csv`

Also download the metadata for later reference.


## Preprocessing the data

### Population data


In [1]:
import pandas as pd
%matplotlib inline

In [2]:
population = pd.read_csv('37259eng_UntypedDataSet_06112024_174233.csv', delimiter=';')

In [3]:
population.head()

,ID,Sex,Regions,Periods,LiveBornChildren_3,NetMigrationIncludingAdministrative_17
0,410,T001038,PV20,1942JJ00,8564.0,-2106.0
1,411,T001038,PV20,1943JJ00,9589.0,-1201.0
2,412,T001038,PV20,1944JJ00,9802.0,-931.0
3,413,T001038,PV20,1945JJ00,9872.0,-2559.0
4,414,T001038,PV20,1946JJ00,12790.0,-3851.0


We first remove all columns that are not necessary.


In [4]:
population['Sex'].value_counts()

Sex
T001038    984
Name: count, dtype: int64

In [5]:
population.drop(['Sex','ID'], axis = 1, inplace = True)

In [6]:
population

,Regions,Periods,LiveBornChildren_3,NetMigrationIncludingAdministrative_17
0,PV20,1942JJ00,8564.0,-2106.0
1,PV20,1943JJ00,9589.0,-1201.0
2,PV20,1944JJ00,9802.0,-931.0
3,PV20,1945JJ00,9872.0,-2559.0
4,PV20,1946JJ00,12790.0,-3851.0
...,...,...,...,...
979,PV31,2019JJ00,8668.0,4450.0
980,PV31,2020JJ00,8652.0,3691.0
981,PV31,2021JJ00,9435.0,6985.0
982,PV31,2022JJ00,8923.0,14195.0


The values of the column Regions contain weird spaces at the end:


In [7]:
population.iloc[0,0]

'PV20  '

We are going to remove them:


In [8]:
population['Regions'] = population['Regions'].map(lambda x: x.strip())

By having a look at the metadata (using CTRL-F for looking for PV20), we can
find out what the province codes actually mean. Let's recode that by using a
dict to map the keys to more meaningful values.


In [9]:
provinces = {"PV20":"Groningen",
"PV21":"Friesland",
"PV22":"Drenthe",
"PV23":"Overijssel",
"PV24":"Flevoland",
"PV25":"Gelderland",
"PV26":"Utrecht",
"PV27":"Noord-Holland",
"PV28":"Zuid-Holland",
"PV29":"Zeeland",
"PV30":"Noord-Brabant",
"PV31":"Limburg"}

In [10]:
population['Regions'] = population['Regions'].map(provinces)

In [11]:
population.head()

,Regions,Periods,LiveBornChildren_3,NetMigrationIncludingAdministrative_17
0,Groningen,1942JJ00,8564.0,-2106.0
1,Groningen,1943JJ00,9589.0,-1201.0
2,Groningen,1944JJ00,9802.0,-931.0
3,Groningen,1945JJ00,9872.0,-2559.0
4,Groningen,1946JJ00,12790.0,-3851.0


Let's also represent the Period in a better way. It's a string now, and only the
first four digits are meaningful. Let's convert these to an integer.
Alternatively, we could opt to convert it to a date (a so-called datetime
object).


In [12]:
population['Periods'] = population['Periods'].map(lambda x: int(x[:4]))

In [13]:
population.head()

,Regions,Periods,LiveBornChildren_3,NetMigrationIncludingAdministrative_17
0,Groningen,1942,8564.0,-2106.0
1,Groningen,1943,9589.0,-1201.0
2,Groningen,1944,9802.0,-931.0
3,Groningen,1945,9872.0,-2559.0
4,Groningen,1946,12790.0,-3851.0


Let's save this:


In [14]:
population.to_csv('population.csv')
population.to_json('population.json')

### Economic data

We just do exactly the same for our economic dataset


In [15]:
economy = pd.read_csv('82800ENG_UntypedDataSet_06112024_174453.csv', delimiter=';')

In [16]:
economy

,ID,EconomicSectorsSIC2008,Regions,Periods,GDPVolumeChanges_1
0,132,T001081,PV20,1996JJ00,9.3
1,133,T001081,PV20,1997JJ00,-2.0
2,134,T001081,PV20,1998JJ00,-0.9
3,135,T001081,PV20,1999JJ00,-0.7
4,136,T001081,PV20,2000JJ00,1.5
...,...,...,...,...,...
259,391,T001081,PV31,2013JJ00,-0.7
260,392,T001081,PV31,2014JJ00,0.2
261,393,T001081,PV31,2015JJ00,2.7
262,394,T001081,PV31,2016JJ00,2.2


In [17]:
# We only downloaded the total, so we can safely delete:
economy['EconomicSectorsSIC2008'].value_counts()

EconomicSectorsSIC2008
T001081       264
Name: count, dtype: int64

In [18]:
economy.drop(['EconomicSectorsSIC2008','ID'], axis = 1, inplace = True)

In [19]:
economy['Regions'] = economy['Regions'].map(lambda x: x.strip())
economy['Regions'] = economy['Regions'].map(provinces)

In [20]:
economy['Periods'] = economy['Periods'].map(lambda x: int(x[:4]))

In [21]:
economy.head()

,Regions,Periods,GDPVolumeChanges_1
0,Groningen,1996,9.3
1,Groningen,1997,-2.0
2,Groningen,1998,-0.9
3,Groningen,1999,-0.7
4,Groningen,2000,1.5


In [22]:
economy.to_csv('economy.csv')
economy.to_json('economy.json')